Now that we have seen some of the differences in performance between the .NET and vSphere objects returned by PowerCLI, we will dig in a bit further to what these objects contain.

This entire vSphere complex object is also contained within the "ExtensionData" property of the .NET object returned from `Get-VM`.

This is unknown to many PowerCLI users, but this is a significant part of why getting the .NET object takes so long to get.

Let's start with connecting to our environment.

In [2]:
$vCenterServer = 'flashstack-vcenter.puretec.purestorage.com'
$vCenterUser= 'administrator@vsphere.local'
$vCenterCred = Get-Credential -UserName $vCenterUser
Connect-VIServer -Server $vCenterServer -Credential $vCenterCred -Protocol https -ErrorAction Stop

Now that we're connected, see how long it takes for us to use `Get-VM` to return a .NET object of a virtual machine.

As this is intended to be an object returned to some expecting to see it as a typical PowerShell object, it will have a predefined set of properties returned.

In [3]:
#Get-VM : .NET Objects
$VMs = Get-VM
$dotNetVM = $VMs | Where-Object { $_.Name -eq 'win-jump' }
$dotNetVM


Name                 PowerState Num CPUs MemoryGB
----                 ---------- -------- --------
win-jump             PoweredOn  2        8.000



We will then take a look at all of the properties returned.

In [4]:
$dotNetVM | Format-List -Property *


Name                    : win-jump
PowerState              : PoweredOn
Notes                   : 
Guest                   : win-jump:Microsoft Windows Server 2019 (64-bit)
NumCpu                  : 2
CoresPerSocket          : 2
MemoryMB                : 8192
MemoryGB                : 8
VMHostId                : HostSystem-host-804089
VMHost                  : fs-metclus1-esxi02.puretec.purestorage.com
VApp                    : 
FolderId                : Folder-group-v56
Folder                  : FS-Joe
ResourcePoolId          : ResourcePool-resgroup-804077
ResourcePool            : Resources
HARestartPriority       : ClusterRestartPriority
HAIsolationResponse     : AsSpecifiedByCluster
DrsAutomationLevel      : AsSpecifiedByCluster
VMSwapfilePolicy        : Inherit
VMResourceConfiguration : CpuShares:Normal/2000 MemShares:Normal/81920
Version                 : Unknown
HardwareVersion         : vmx-17
PersistentId            : 5029ac95-568f-0c9b-8025-696f254c5050
GuestId               

Finally, we will open up the snapshot information about the virtual machines, which we will find under the `ExtensionData` property.

We will compare this to our vSphere object in a few steps:

In [5]:
$VMs | Select-Object Name, @{n = 'Snapshot'; e = {($_.ExtensionData.Layout.Snapshot).SnapshotFile}} | `
Where-Object Snapshot


Name                  Snapshot
----                  --------
orc-node2             {[FlashStack-Prod] orc-node2/orc-node2-Snapshot1.vmsn, [FlashStack-Prod] orc…
orc-node1             {[FlashStack-Prod] orc-node1/orc-node1-Snapshot1.vmsn, [FlashStack-Prod] orc…
postgresdb-test       {[FlashStack-Prod] postgresdb-test_1/postgresdb-test-Snapshot1.vmsn, [FlashS…
fs-victoriametricslab {[FlashStack-Prod] fs-victoriametricslab/fs-victoriametricslab-Snapshot1.vms…
fs-prometheuslab      {[FlashStack-Prod] fs-prometheuslab/fs-prometheuslab-Snapshot1.vmsn, [FlashS…
FlashStack-VC         {[FlashStack-Prod] FlashStack-VC/FlashStack-VC-Snapshot1.vmsn, [FlashStack-P…
intersight-app        {[Metro_Cluster1_Prod02] intersight-app/intersight-app-Snapshot1.vmsn, [Metr…
fs-linjmp01           {[FlashStack-Misc] fs-linjmp01/fs-linjmp01-Snapshot1.vmsn, [FlashStack-Misc]…
FS-WinJmp1            {[FlashStack-Misc] FS-WinJmp1/FS-WinJmp1-Snapshot1.vmsn, [FlashStack-Misc] F…
fs-vidm               {[FlashStack-Pr

Now that we have looked at our .NET object, we will use `Get-View` to return a vSphere object of a virtual machine, which essentially gives us our API view of the object.

We will see that the properties around snapshots match the details from `Get-VM`, but with much better performance.

In [6]:
#Get-View : vSphere Managed Object
$ViewVMs = Get-View -ViewType VirtualMachine
$ViewVM = $ViewVMs | Where-Object { $_.Name -eq 'win-jump' }
$ViewVM


Capability           : VMware.Vim.VirtualMachineCapability
Config               : VMware.Vim.VirtualMachineConfigInfo
Layout               : VMware.Vim.VirtualMachineFileLayout
LayoutEx             : VMware.Vim.VirtualMachineFileLayoutEx
Storage              : VMware.Vim.VirtualMachineStorageInfo
EnvironmentBrowser   : EnvironmentBrowser-envbrowser-803039
ResourcePool         : ResourcePool-resgroup-804077
ParentVApp           : 
ResourceConfig       : VMware.Vim.ResourceConfigSpec
Runtime              : VMware.Vim.VirtualMachineRuntimeInfo
Guest                : VMware.Vim.GuestInfo
Summary              : VMware.Vim.VirtualMachineSummary
Datastore            : {Datastore-datastore-804098}
Network              : {DistributedVirtualPortgroup-dvportgroup-64}
Snapshot             : 
RootSnapshot         : {}
GuestHeartbeatStatus : green
LinkedView           : 
Parent               : Folder-group-v56
CustomValue          : {}
OverallStatus        : green
ConfigStatus         : green
Confi

We will then take a look at all of the properties returned.

In [7]:
$ViewVM | Format-List -Property *


Capability           : VMware.Vim.VirtualMachineCapability
Config               : VMware.Vim.VirtualMachineConfigInfo
Layout               : VMware.Vim.VirtualMachineFileLayout
LayoutEx             : VMware.Vim.VirtualMachineFileLayoutEx
Storage              : VMware.Vim.VirtualMachineStorageInfo
EnvironmentBrowser   : EnvironmentBrowser-envbrowser-803039
ResourcePool         : ResourcePool-resgroup-804077
ParentVApp           : 
ResourceConfig       : VMware.Vim.ResourceConfigSpec
Runtime              : VMware.Vim.VirtualMachineRuntimeInfo
Guest                : VMware.Vim.GuestInfo
Summary              : VMware.Vim.VirtualMachineSummary
Datastore            : {Datastore-datastore-804098}
Network              : {DistributedVirtualPortgroup-dvportgroup-64}
Snapshot             : 
RootSnapshot         : {}
GuestHeartbeatStatus : green
LinkedView           : 
Parent               : Folder-group-v56
CustomValue          : {}
OverallStatus        : green
ConfigStatus         : green
Confi

Finally, we will open up the snapshot information about the virtual machines.

If we pay attention, we will see that the properties around snapshots are the same against both the .NET and vSphere objects.

In [8]:
$ViewVMs | Select-Object Name, @{n = 'Snapshot'; e = {($_.Layout.Snapshot).SnapshotFile}} | `
Where-Object Snapshot


Name                  Snapshot
----                  --------
intersight-app        {[Metro_Cluster1_Prod02] intersight-app/intersight-app-Snapshot1.vmsn, [Metr…
fslab-ad              {[FlashStack_ISO_ContLib] fslab-ad/fslab-ad-Snapshot1.vmsn, [FlashStack_ISO_…
FS-WinJmp1            {[FlashStack-Misc] FS-WinJmp1/FS-WinJmp1-Snapshot1.vmsn, [FlashStack-Misc] F…
postgresdb-test       {[FlashStack-Prod] postgresdb-test_1/postgresdb-test-Snapshot1.vmsn, [FlashS…
fs-vidm               {[FlashStack-Prod] fs-vidm/fs-vidm-Snapshot1.vmsn, [FlashStack-Prod] fs-vidm…
fs-vra                {[FlashStack-Prod] fs-vra/fs-vra-Snapshot1.vmsn, [FlashStack-Prod] fs-vra/fs…
fs-vrlscm             {[FlashStack-Misc] fs-vrlscm/fs-vrlscm-Snapshot2.vmsn, [FlashStack-Misc] fs-…
rheltemplate-fslab    {[FlashStack-Prod] rheltemplate-fslab/rheltemplate-fslab-Snapshot1.vmsn, [Fl…
FlashStack-VC         {[FlashStack-Prod] FlashStack-VC/FlashStack-VC-Snapshot1.vmsn, [FlashStack-P…
fs-linjmp01           {[FlashStack-Mi

To see all view types that exist, you can type `Get-View -ViewType ` and then use tab completion to see all of the view types which are available to select.